In [1]:
import os

os.chdir('/content/')
!(echo 'Clone LongSeq git ...'; git clone https://github.com/AUTH-MINT/LongSeq.git; echo 'done')
!test -d /content/ebm_nlp/ || (echo 'Download EBM-NLP dataset'; git clone https://github.com/bepnye/EBM-NLP.git ebm_nlp; echo 'done')

os.chdir('/content/LongSeq/')
!test -d /content/LongSeq/ebm_nlp_2_00/ || (echo 'Extracting documents and labels...'; tar -xzf /content/ebm_nlp/ebm_nlp_2_00.tar.gz; echo 'done')

os.chdir('/content/')
!(echo 'Delete unnecessary files...'; rm -rf /content/ebm_nlp/; echo 'done')
print('All ready!')

Create content file.
Clone LongSeq git ...
Cloning into 'LongSeq'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 99 (delta 34), reused 55 (delta 10), pack-reused 0
Unpacking objects: 100% (99/99), done.
done
Download EBM-NLP dataset
Cloning into 'ebm_nlp'...
remote: Enumerating objects: 200, done.
remote: Total 200 (delta 0), reused 0 (delta 0), pack-reused 200
Receiving objects: 100% (200/200), 83.47 MiB | 21.00 MiB/s, done.
Resolving deltas: 100% (82/82), done.
done
Extracting documents and labels...
done
Delete unnecessary files...
done
All ready!


In [ ]:
print("Installing transformers...\n")
!pip install transformers==3.3.1
print("\n")
print("Installing pytorch crf...\n")
!pip install pytorch-crf
print("\n")

Installing transformers...

     |████████████████████████████████| 1.1 MB 5.1 MB/s 
     |████████████████████████████████| 3.0 MB 36.6 MB/s 
     |████████████████████████████████| 895 kB 34.6 MB/s 
     |████████████████████████████████| 1.2 MB 36.3 MB/s 


Installing pytorch crf...





In [ ]:
os.chdir('/content/LongSeq/')
import torch
import numpy as np
import os
#import nltk
import build_data
import build_data_abstracts
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import time
import model.dataset as dataset
import model.engine as engine
from model.modeltr import EntityModel
from model.data_utils import CoNLLDataset, data_to_dict
from model.config import Config
from model.utils import timeSince, metrics
import model.utils as utils

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/228k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [ ]:
config = Config()
 
data_prefix = 'p1_all'
 
print("Check if files exist otwerwise create them!")

print("Use sentences:",config.use_sen)
 
if config.use_sen:
    print("Dataset Contains each sentence.")
    print('Generating tagged and formatted data for the model')
    build_data.generate_model_data(data_prefix)
    print('Finished. Ready to train model on %s' %data_prefix)
    cwd = os.getcwd()
    config.filename_dev = os.path.join(cwd, 'data', data_prefix + '_' + os.path.basename(config.filename_dev))
    config.filename_test = os.path.join(cwd, 'data', data_prefix + '_' + os.path.basename(config.filename_test))
    config.filename_train = os.path.join(cwd, 'data', data_prefix + '_' + os.path.basename(config.filename_train))
    in_tp = 'Sentences'
 
else:
    print("Dataset Contains whole abstract.")
    print('Generating tagged and formatted data for the model')
    build_data_abstracts.generate_model_data(data_prefix)#, voc_build=False)
    print('Finished. Ready to train model on %s' % data_prefix)
    cwd = os.getcwd()
    config.filename_dev = os.path.join(cwd, 'data', data_prefix + '_' + os.path.basename(config.filename_dev_abs))
    config.filename_test = os.path.join(cwd, 'data', data_prefix + '_' + os.path.basename(config.filename_test_abs))
    config.filename_train = os.path.join(cwd, 'data', data_prefix + '_' + os.path.basename(config.filename_train_abs))
    in_tp = "Abstract"
 
print("All done...")

Check if files exist otwerwise create them!
Use sentences: False
Dataset Contains whole abstract.
Generating tagged and formatted data for the model
Config
Processing_word
Preprocessing tokens and labels to generate input data files
Reading files for starting_spans participants
Reading files for starting_spans interventions
Reading files for starting_spans outcomes
Reading files for hierarchical_labels participants
Reading files for hierarchical_labels interventions
Reading files for hierarchical_labels outcomes
	4792 ids for train
	189 ids for gold
	4782 ids for train
	188 ids for gold
	4670 ids for train
	190 ids for gold
Found 4794 ids for train
Found 191 ids for gold
Using 4794 ids for train
Split training set in to 3836 train, 958 dev
	4609 ids for train
	189 ids for gold
Found 4609 ids for train
Found 189 ids for gold
Using 4609 ids for train
Split training set in to 3688 train, 921 dev
	4746 ids for train
	187 ids for gold
Found 4746 ids for train
Found 187 ids for gold
Using 47

In [ ]:
parameters = {
    'Use:':in_tp,
    'Pretrained Language Model:':Config.PRETRAINED_MODEL,
    'Number of epochs:':Config.nepochs,
    'Learning Rate:':Config.lr,
    'N:':Config.N,
    'h:':Config.h,
    'd_model:':Config.d_model,
    'Train_Batch_Size':Config.train_batch_size,
    'Test_Batch_Size':Config.valid_batch_size
}

print("Parameters:\n")
for i in parameters:
    print(i,parameters[i])


Parameters:

Use: Abstract
Pretrained Language Model: allenai/scibert_scivocab_uncased
Number of epochs: 6
Learning Rate: 3e-05
N: 4
h: 4
d_model: 768
Train_Batch_Size 4
Test_Batch_Size 4


In [ ]:
# create datasets
print("Load datasets:")

dev = CoNLLDataset(config.filename_dev, config.max_iter)
      
train = CoNLLDataset(config.filename_train, config.max_iter)
      
test  = CoNLLDataset(config.filename_test, config.max_iter)
      

Load datasets:


In [ ]:
print('Transform data to dictionaries:')
devDict = data_to_dict(dev)
trainDict = data_to_dict(train)
testDict = data_to_dict(test)
num_tag = len(list(utils.enc_trans.classes_))

Transform data to dictionaries:


In [ ]:
print(devDict['words'][0])

['Bioequivalence', 'evaluation', 'of', 'a', 'fixed', 'dose', 'combination', 'lamivudine', '+', 'stavudine', 'tablet', 'with', 'concurrent', 'administration', 'of', 'lamivudine', 'tablet', 'and', 'stavudine', 'capsule', 'in', 'healthy', 'volunteers', '.', 'The', 'study', 'was', 'designed', 'to', 'compare', 'the', 'rate', 'and', 'extent', 'of', 'absorption', 'of', 'a', 'fixed', 'dose', 'combination', 'tablet', 'of', 'lamivudine', '(', 'CAS', '134678-17-4', ')', 'and', 'stavudine', '(', 'CAS', '3056-17-5', ')', 'with', 'the', 'concurrent', 'administration', 'of', 'lamivudine', 'tablet', 'and', 'stavudine', 'capsule', 'in', '24', 'healthy', 'volunteers', 'under', 'fasting', 'conditions', '.', 'The', 'volunteers', 'were', 'randomly', 'assigned', 'to', 'the', 'test', 'or', 'reference', 'treatment', ',', 'with', 'the', 'two', 'treatment', 'periods', 'separated', 'by', 'a', 'washout', 'period', 'of', 'at', 'least', '7', 'days', '.', 'Plasma', 'samples', 'were', 'analyzed', 'for', 'both', 'anal

In [ ]:
print('Initialize dataloaders:')

train_dataset = dataset.EntityDataset(
        words=trainDict['words'], tags=trainDict['enc_tags'], config=config
    )

train_data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=Config.train_batch_size, num_workers=4
    )

valid_dataset = dataset.EntityDataset(
        words=devDict['words'], tags=devDict['enc_tags'], config=config
    )

valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=Config.valid_batch_size, num_workers=1
    )

test_dataset = dataset.EntityDataset(
        words=testDict['words'], tags=testDict['enc_tags'], config=config
    )

test_data_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=1, num_workers=1
    )


Initialize dataloaders:


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
device = torch.device("cuda")
model = EntityModel(num_tag=num_tag, tr=True)
model.to(device)

EntityModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)


In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.001,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]

In [ ]:
num_train_steps = int(len(trainDict['words']) / Config.train_batch_size * Config.nepochs)
optimizer = AdamW(optimizer_parameters, lr=Config.lr)
scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)

In [ ]:
print("Training starts...\n\n")
start = time.time()

best_loss = np.inf
for epoch in range(Config.nepochs):
    print("Epoch %g from %g"%(epoch+1,Config.nepochs))
    train_loss = engine.train_fn(train_data_loader, model, optimizer, device, scheduler)
    test_loss = engine.eval_fn(valid_data_loader, model, device)
    print(f"Train Loss = {train_loss} Valid Loss = {test_loss}")
    if test_loss < best_loss:
        torch.save(model.state_dict(), Config.MODEL_PATH)
        best_loss = test_loss
        print("\nNew best score!\n")

total_time = timeSince(start)

print('Total training time:',total_time)
parameters["Training Time:"] = total_time

Training starts...


Epoch 1 from 6


  0%|          | 0/959 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torchcrf/__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  /pytorch/aten/src/ATen/native/TensorCompare.cpp:255.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)
100%|██████████| 240/240 [02:38<00:00,  1.51it/s]


Train Loss = 640.2417553532732 Valid Loss = 559.2056945800781

New best score!

Epoch 2 from 6


100%|██████████| 240/240 [02:43<00:00,  1.47it/s]


Train Loss = 497.30337285746873 Valid Loss = 548.8421152750651

New best score!

Epoch 3 from 6


...

Train Loss = 347.267991560219 Valid Loss = 601.8424270629882
Epoch 5 from 6


100%|██████████| 240/240 [02:39<00:00,  1.50it/s]


Train Loss = 294.7894303656967 Valid Loss = 636.6905766805013
Epoch 6 from 6


100%|██████████| 240/240 [02:43<00:00,  1.47it/s]

Train Loss = 260.67704319481555 Valid Loss = 674.5224395751953
Total training time: 168m 33s


In [ ]:
print("Testing...\n")

model = EntityModel(num_tag=num_tag, tr=False)
model.load_state_dict(torch.load(Config.MODEL_PATH))
model.to(device)
results = engine.test_fn(test_data_loader, model, device)

y = results['y']
y_pred = results['y_pred']


print("Labels, no X")
res3 = metrics(y,y_pred, labels=utils.def_tags[:-1], target_names=utils.labels)
print("Parameters:\n")
for i in parameters:
    print(i,parameters[i])

Testing...



100%|██████████| 190/190 [00:30<00:00,  6.23it/s]


Labels, no X


Classification Report:

              precision    recall  f1-score   support

 Intervation       0.72      0.78      0.75      5802
     Outcome       0.84      0.61      0.70      9451
     Patient       0.82      0.85      0.84      5487
        None       0.87      0.91      0.89     39573

   micro avg       0.85      0.85      0.85     60313
   macro avg       0.81      0.79      0.79     60313
weighted avg       0.85      0.85      0.84     60313



